In [6]:
import numpy as np
import cv2
from keras.preprocessing import image
import time
import align.detect_face
import os

import tensorflow as tf
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.models import model_from_json


In [7]:
# facenet 
image_size=160
margin= 44
gpu_memory_fraction=1.0

def load_and_align_data(img, image_size,margin, gpu_memory_fraction):
    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
    factor = 0.709 # scale factor
    with tf.Graph().as_default():
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
        with sess.as_default():
            pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
    #img = scipy.misc.imread(os.path.expanduser(image_path))
    img_size = np.asarray(img.shape)[0:2]
    bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
    if (len(bounding_boxes)==0):
        bb=0
        have_face = 0
    else:
        det = np.squeeze(bounding_boxes[0,0:4])
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(det[0]-margin/2, 0)
        bb[1] = np.maximum(det[1]-margin/2, 0)
        bb[2] = np.minimum(det[2]+margin/2 - bb[0], img_size[1])
        bb[3] = np.minimum(det[3]+margin/2 - bb[1], img_size[0])
        have_face = 1
    return bb,have_face

In [8]:
basedir = "/home/topica/Video_topica"
cropdir = "/home/topica/crop_face_topica"

In [9]:
cap = cv2.VideoCapture(os.path.join(basedir,"T6_08_20180816_100000.avi"))
# cap = cv2.VideoCapture("/home/topica/Video_topica/T6_08_20180816_090000.avi")

frame = 0

In [5]:
while(True):
    ret, img = cap.read()
    cap.set(cv2.CAP_PROP_POS_MSEC,(frame*300))   
    detect_face, have_face= load_and_align_data(img,image_size,margin,gpu_memory_fraction)
    if (have_face!=0):
        detect_face = np.reshape(detect_face,(-1,4)) 
        for (x,y,w,h) in detect_face:
            detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
        cv2.imwrite(os.path.join(cropdir,"face_5_"+str(frame)+".png"),detected_face)
#     cv2.imshow("img", img)
    frame = frame + 1
    if cv2.waitKey(70) & 0xFF == ord('q'): #press q to quit
        break
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 